In [ ]:
# Code to mount the drive 
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# mapping_csv = pd.read_csv('/content/drive/MyDrive/685/emogen/Classifier/train/caption/train_caption.csv')

In [ ]:
# %cd /content/drive/MyDrive/685/emogen/Classifier/
# !unzip train.zip

In [ ]:
!pip install -U sentence-transformers

In [ ]:
cd '/content/drive/MyDrive/685/catr'

/content/drive/.shortcut-targets-by-id/12c1zkm0_oa8VcOfsUa8Tn_YcYdyPpSlP/685/catr


In [ ]:
!pip install -r requirements.txt

In [ ]:
# from transformers import ViTModel, ViTConfig, ViTFeatureExtractor,BertTokenizer,BertForMaskedLM
from transformers import DeiTFeatureExtractor, DeiTModel #AutoFeatureExtractor, DeiTForImageClassificationWithTeacher, 
from sentence_transformers import SentenceTransformer
import torch
from sklearn.model_selection import KFold
from sklearn.metrics import auc, precision_score, recall_score,roc_auc_score
import xgboost as xgb
from PIL import Image
import argparse
import glob
import json
import pandas as pd
import numpy as np
from models import caption
from datasets import coco, utils
from tqdm import tqdm
from configuration import Config
from xgboost import XGBClassifier
import os

In [ ]:
def extract_vision_transformer_feats(image_path,nsel_st=None,nsel_end=None):
  """
  Function to extract the features from vision transformers
  """
  if not nsel_st:
    img_files_list = glob.glob(image_path+"*")
  else:
    img_files_list = glob.glob(image_path+"*")[nsel_st:nsel_end]
  
  # Create image batch array for Vision Transformer
  img_batch = []
  for file in tqdm(img_files_list):
    img = np.asarray(Image.open(file))
    newsize = (240, 240, 3)
    img = np.resize(img,newsize)
    img_batch.append(img)
  print("Creation of image batches to be used for Vision Transformer complete")

  
  # Extract features from ViTModel
  # feature_extractor   = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
  # model               = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
  feature_extractor   =  DeiTFeatureExtractor.from_pretrained('facebook/deit-tiny-distilled-patch16-224')
  model               = DeiTModel.from_pretrained('facebook/deit-tiny-distilled-patch16-224')
  inputs              = feature_extractor(images=img_batch, return_tensors="pt")
  outputs             = model(**inputs)

  # Get image representations and their corresponding hashes i.e. get [CLS] token representation for each image
  img_representations = outputs.last_hidden_state[:,0,:]
  img_hash_li         = []
  for file in img_files_list:
    img_hash_li.append(file.split("/")[-1][:-4])
  
  # Create column names for image dimensions 
  col_img  = ["imdim_"+str(i) for i in list(range(192))]

  # Create a dataframe of image features
  img_data = pd.DataFrame(img_representations.detach().numpy(),columns = col_img)
  img_data['img_hash'] = img_hash_li

  return img_data

def extract_sentence_transformer_feats(reference_file_pth = '/content/drive/MyDrive/685/sarcasm/train_data/caption/',csv_file = 'train_caption.csv'):
  """
  Extract features for emotion related texts
  """
  # Get reference text data
  combined_path = reference_file_pth + csv_file
  #file = open(combined_path)
  #data_json = json.load(file)
  data = pd.read_csv(combined_path)#pd.DataFrame.from_dict(data_json['annotations']).reset_index(drop=True)
  try:
    del data['Unnamed: 0']
  except:
    pass
  
  # Use Sentence Transformer to extract features
  model = SentenceTransformer('all-mpnet-base-v2')
  sentence_embeddings = model.encode(data['comment'])
  col_text = ["tdim_"+str(i) for i in list(range(768))]

  # Create text feature dataframe
  text_data = pd.DataFrame(sentence_embeddings,columns=col_text)
  text_data['img_hash'] = data['image_hash']
  text_data['label'] = data['label']

  return text_data

def xgb_train_kfold(X_trn, y_trn,n_splits=5,max_depth=3,n_estimator=150, rand_st=3815):
  """
  Perform training with XGBoost and evaluate in K-Fold cross-validation settings 
  """
  errors    = []
  precision = []
  recall    = []
  auc       = []
  kf = KFold(n_splits=n_splits, shuffle=True, random_state=3815)

  for train_index, test_index in tqdm(kf.split(X_trn)):
     X_train_n, X_test_n = X_trn.values[train_index], X_trn.values[test_index]
     y_train_n, y_test_n = y_trn.values[train_index], y_trn.values[test_index]

     model = XGBClassifier(
         max_depth=max_depth, n_estimators=n_estimator, random_state = rand_st
     )
     model.fit(X_train_n, y_train_n)
     y_pred = model.predict(X_test_n)
     accuracy = (sum(y_pred == y_test_n))/len(y_test_n)
     errors.append(1 - accuracy)
     precision.append(precision_score(y_test_n,y_pred))
     recall.append(recall_score(y_test_n,y_pred))
     auc.append(roc_auc_score(y_test_n,y_pred))

  return errors, precision, recall, auc

In [ ]:
img_data = extract_vision_transformer_feats(image_path= '/content/drive/MyDrive/685/sarcasm/train/',nsel_st=1400,nsel_end=1600)

100%|██████████| 200/200 [00:01<00:00, 111.69it/s]


Creation of image batches to be used for Vision Transformer complete


Some weights of the model checkpoint at facebook/deit-tiny-distilled-patch16-224 were not used when initializing DeiTModel: ['distillation_classifier.bias', 'distillation_classifier.weight', 'cls_classifier.weight', 'cls_classifier.bias']
- This IS expected if you are initializing DeiTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DeiTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DeiTModel were not initialized from the model checkpoint at facebook/deit-tiny-distilled-patch16-224 and are newly initialized: ['deit.pooler.dense.weight', 'deit.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference

In [ ]:
img_data.to_csv('/content/drive/MyDrive/685/sarcasm/evaluation/img_feats_1600_batch.csv')

In [ ]:
text_data = extract_sentence_transformer_feats()

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

KeyError: ignored

In [ ]:
mmodal_data = img_data.merge(text_data,how='outer',on='img_hash')
#print(mmodal_data.columns)
labels = [int(x) for x in mmodal_data['label']]
# Create pseudo labels for interim 
#pseudo_labels = [np.random.randint(0,2) for i in range(len(mmodal_data))]
mmodal_data['Target']  = labels
###########################################################
y_trn = mmodal_data['Target']
X_trn = mmodal_data.drop(columns=['label','Target','img_hash'],axis=1)

In [ ]:
errors, precision, recall, auc = xgb_train_kfold(X_trn, y_trn,n_splits=5)

In [ ]:
# Check average performances across K-Folds
print("The training errors on average is: ", np.round(np.mean(errors),4)*100)
print("The Precision on average is: ", np.round(np.mean(precision),4)*100)
print("The Recall on average is: ", np.round(np.mean(recall),4)*100)
print("The AUC Score on average is: ", np.round(np.mean(auc),4)*100)

In [ ]:
text_data = extract_sentence_transformer_feats(reference_file_pth = '../emogen/annotations/',json_file = 'captions_train.json')
# Concat data to for final dataset with multimodal features  
mmodal_data = img_data.merge(text_data,how='left',on='img_hash')
# Create pseudo labels for interim 
pseudo_labels = [np.random.randint(0,2) for i in range(len(mmodal_data))]
mmodal_data['Target']  = pseudo_labels
###########################################################
y_trn = mmodal_data['Target']
X_trn = mmodal_data.drop(columns=['Target','img_hash'],axis=1)
errors, precision, recall, auc = xgb_train_kfold(X_trn,y_trn)
# Check average performances across K-Folds
print("The training errors on average is: ", np.round(np.mean(errors),4)*100)
print("The Precision on average is: ", np.round(np.mean(precision),4)*100)
print("The Recall on average is: ", np.round(np.mean(recall),4)*100)
print("The AUC Score on average is: ", np.round(np.mean(auc),4)*100)

The training errors on average is:  60.0
The Precision on average is:  36.69
The Recall on average is:  34.33
The AUC Score on average is:  42.75


In [ ]:
# def xgb_train_kfold(X_trn, y_trn,n_splits=5):
#   """
#   Perform training with XGBoost and evaluate in K-Fold cross-validation settings 
#   """
#   errors    = []
#   precision = []
#   recall    = []
#   auc       = []
#   kf = KFold(n_splits=n_splits, shuffle=True, random_state=3815)

#   for train_index, test_index in kf.split(X_trn):
#      X_train_n, X_test_n = X_trn.values[train_index], X_trn.values[test_index]
#      y_train_n, y_test_n = y_trn.values[train_index], y_trn.values[test_index]

#      model = XGBClassifier(
#          max_depth=4, n_estimators=100, random_state = 3815
#      )
#      model.fit(X_train_n, y_train_n)
#      y_pred = model.predict(X_test_n)
#      accuracy = (sum(y_pred == y_test_n))/len(y_test_n)
#      errors.append(1 - accuracy)
#      precision.append(precision_score(y_test_n,y_pred))
#      recall.append(recall_score(y_test_n,y_pred))
#      auc.append(roc_auc_score(y_test_n,y_pred))

#   return errors, precision, recall, auc


# Assign image folder, image path and mmodel checkpoint path 
# image_path = '../emogen/train/'
# #img_name   = '0c4ff62871a904274fd41cee695d85f.jpg'
# #img_final_loc = image_path+img_name
# # Load image and perform pre-processing 
# #image = Image.open(img_final_loc)

# img_files_list = glob.glob(image_path+"*")[:50] # For test run 

# img_batch = []
# for file in tqdm(img_files_list):
#   img = np.asarray(Image.open(file))
#   img_batch.append(img)

# # Extract features
# feature_extractor   = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
# model               = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
# inputs              = feature_extractor(images=img_batch, return_tensors="pt")
# outputs             = model(**inputs)

# # Get image representations and their corresponding hashes 
# img_representations = outputs.last_hidden_state[:,0,:]
# img_hash_li         = []
# for file in img_files_list:
#   img_hash_li.append(file.split("/")[-1][:-4])

# # Get reference text data
# reference_file_pth = '../emogen/annotations/'
# json_file = 'captions_train.json'
# combined_path = reference_file_pth + json_file

# file = open(combined_path)
# data_json = json.load(file)
# data = pd.DataFrame.from_dict(data_json['annotations']).reset_index(drop=True)
# try:
#   del data['id']
# except:
#   pass
# model = SentenceTransformer('all-mpnet-base-v2')
# sentence_embeddings = model.encode(data['caption'])

# # Create column names 
# col_text = ["tdim_"+str(i) for i in list(range(768))]
# col_img  = ["imdim_"+str(i) for i in list(range(768))]
# # Create DataFrame for text features  
# text_data = pd.DataFrame(sentence_embeddings,columns=col_text)
# text_data['img_hash'] = data['image_id']
# # Create a dataframe of image features
# img_data = pd.DataFrame(img_representations.detach().numpy(),columns = col_img)
# img_data['img_hash'] = img_hash_li

# # Concat data to for final dataset with multimodal features  
# mmodal_data = img_data.merge(text_data,how='left',on='img_hash')

# # Create pseudo labels for interim 
# pseudo_labels = [np.random.randint(0,2) for i in range(len(mmodal_data))]
# mmodal_data['Target']  = pseudo_labels

# y_trn = mmodal_data['Target']
# X_trn = mmodal_data.drop(columns=['Target','img_hash'],axis=1)

# errors, precision, recall, auc = xgb_train_kfold(X_trn,y_trn)

# # Check average performances across K-Folds
# print("The training errors on average is: ", np.round(np.mean(errors),4)*100)
# print("The Precision on average is: ", np.round(np.mean(precision),4)*100)
# print("The Recall on average is: ", np.round(np.mean(recall),4)*100)
# print("The AUC Score on average is: ", np.round(np.mean(auc),4)*100)

In [ ]:
!nvidia-smi

Sat Dec 11 10:57:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------